In [101]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import os

In [102]:
os.makedirs('./output_csv', exist_ok=True)
os.makedirs('./input_csv', exist_ok=True)

In [103]:
df_bloomberg = pd.DataFrame()

for year in range(2000, 2026):
    try:
        df_year = pd.read_csv(f"./input_csv/bloomberg_raw/{year}.csv")
        df_bloomberg = pd.concat([df_bloomberg, df_year])
    except FileNotFoundError:
        continue

In [104]:
df_bloomberg['Filing Term Price Range'] = df_bloomberg['Filing Term Price Range'].str.replace('\n', '').str.strip()
df_bloomberg['Trade Date (US)'] = pd.to_datetime(df_bloomberg['Trade Date (US)'], format='%m/%d/%y', errors='coerce')
df_bloomberg['Trade Date (US) FoM'] = df_bloomberg['Trade Date (US)'].dt.to_period('M').dt.to_timestamp()

In [105]:
def clean_macroeconomic_data(file_name):
    tmp = pd.read_csv(file_name)
    tmp['observation_date'] = pd.to_datetime(tmp['observation_date']).dt.to_period('M').dt.to_timestamp()
    tmp = tmp.groupby('observation_date').mean().reset_index()
    tmp.rename(columns={'observation_date': 'Trade Date (US) FoM'}, inplace=True)
    tmp['Trade Date (US) FoM'] = pd.to_datetime(tmp['Trade Date (US) FoM'], errors='coerce')
    return tmp

In [106]:
df_dff = clean_macroeconomic_data('./input_csv/DFF.csv')
df_cpi = clean_macroeconomic_data('./input_csv/CPI.csv')
df_csent = clean_macroeconomic_data('./input_csv/UMCSENT.csv')
df_unrate = clean_macroeconomic_data('./input_csv/UNRATE.csv')

In [107]:
df_bloomberg = (df_bloomberg.merge(df_dff, 
                                  on='Trade Date (US) FoM', 
                                  how='left')
                            .merge(df_cpi, 
                                  on='Trade Date (US) FoM', 
                                  how='left')
                            .merge(df_csent, 
                                  on='Trade Date (US) FoM', 
                                  how='left')
                            .merge(df_unrate, 
                                  on='Trade Date (US) FoM', 
                                  how='left'))

In [108]:
df_bloomberg.rename(columns={'CPILFESL': 'CPI',
                              'DFF': 'Fed Rate',
                              'UMCSENT': 'Consumer Confidence',
                              'UNRATE': 'Unemployment Rate'
                             }, inplace=True)

In [109]:
df_bloomberg.drop(columns=['Trade Date (US) FoM'], inplace=True)

In [110]:
df_bloomberg.to_csv('./output_csv/bloomberg_data.csv', index=False)
